# Trabajo práctico 5 - Template Matching

**Alumnos:**

- Carol lugones Ignacio (100073)
- Torresetti Lisandro (99846)

## Objetivo

A partir del patrón de la cara de Messi (el cual debe ser seleccionado de entre las imágenes) encontrarlo en cada una de las imágenes en las que Messi aparece. Se pide un algoritmo automático que frente a la lectura sucesiva de estas imágenes (y si quieren, pueden probar con imágenes que encuentren ustedes) devuelva la posición de la cara de Messi dentro de la imagen y el nivel de confianza con el que fue hallada (deben definir también el nivel de confianza propuesto).

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
%matplotlib inline

In [ ]:
def plotter(image, title = '', imgSize = (18,9), grayScale = False, step = 25): #Funcion auxiliar para realizar los graficos
    plt.figure(figsize=imgSize)
    plt.title(title, fontsize = 16, fontweight = "bold")
    plt.imshow(image) if not grayScale else plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    plt.yticks(np.arange(0, len(image), step))
    plt.xticks(np.arange(0, len(image[0]), step))
    plt.show()  

In [ ]:
#Cargamos las imagenes a analizar junto con el patron
pattern = None
patternGray = None
imgsMessi = []
imgsMessiGray = []
imgNames = glob('./Fotos/*') #Cargamos todas las imagenes que se encuentran en el folder 'Fotos'
for name in imgNames:
    img = cv.imread(name)
    imgGray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    if 'Template' in name:
        pattern = img
        patternGray = imgGray
    else:
        imgsMessi.append(img)
        imgsMessiGray.append(imgGray)
    print("Img Name: {}\tImg Width: {}\tImg Height: {}".format(name, img.shape[1], img.shape[0]))

In [ ]:
# Como patrón elegimos la siguiente cara de Messi
plotter(pattern, 'Template')
plotter(patternGray, 'Template in Gray Scale', grayScale=True)

Al cargar las imágenes se puede apreciar que todas tienen distintas resoluciones, debido a esto lo que se hizo fue realizar un análisis a distintas resoluciones para ver si se lograba encontrar con éxito la cara de Messi en esas imágenes. Para lograr esto se utilizaron las funciones brindadas por la cátedra, que se cargan a continuación.

In [ ]:
def gaussian_pyramid(img, num_levels):
    lower = img.copy()
    gaussian_pyr = [lower]
    for i in range(num_levels):
        lower = cv.pyrDown(lower)
        gaussian_pyr.append(np.float32(lower))
    return gaussian_pyr

def laplacian_pyramid(gaussian_pyr):
    laplacian_top = gaussian_pyr[-1]
    num_levels = len(gaussian_pyr) - 1
    
    laplacian_pyr = [laplacian_top]
    for i in range(num_levels,0,-1):
        size = (gaussian_pyr[i - 1].shape[1], gaussian_pyr[i - 1].shape[0])
        gaussian_expanded = cv.pyrUp(gaussian_pyr[i], dstsize=size)
        laplacian = np.subtract(gaussian_pyr[i-1], gaussian_expanded)
        laplacian_pyr.append(laplacian)
    return laplacian_pyr

def getLaplacian(img, desiredLevel):
    return laplacian_pyramid(gaussian_pyramid(img, desiredLevel))

def reconstruct(laplacian_pyr):
    laplacian_top = laplacian_pyr[0]
    laplacian_lst = [laplacian_top]
    num_levels = len(laplacian_pyr) - 1
    for i in range(num_levels):
        size = (laplacian_pyr[i + 1].shape[1], laplacian_pyr[i + 1].shape[0])
        laplacian_expanded = cv.pyrUp(laplacian_top, dstsize=size)
        laplacian_top = cv.add(laplacian_pyr[i+1], laplacian_expanded)
        laplacian_lst.append(laplacian_top)
    return laplacian_lst

In [ ]:
level = 2 # Hasta que nivel de la piramide Gaussiana queremos llegar
selected = -1 # Con que nivel nos quedamos
imgTemplate = gaussian_pyramid(cv.imread("./Fotos/Template.jpg", 0), level)[selected]

In [ ]:
def pyrUp(lowerPoint, originalImg, levelsDone = level):
    laplacian = getLaplacian(originalImg, levelsDone)
    actualCompression = laplacian[1] + cv.pyrUp(lowerPoint)
    for i in range(2, len(laplacian)):
        plotter(actualCompression)
        actualCompression = cv.pyrUp(actualCompression) + laplacian[i]
    return actualCompression

A continuación aplicamos distintos métodos para realizar la comparación de imágenes, cada método lo aplicamos a cada una de las imágenes que tenemos de Messi. El fin de hacer esto es ver con qué método se obtienen mejores resultados.

In [ ]:
#Se corre el codigo de la practica modificado para ver como lo muestra, saltear celda si no interesa el trabajo de verlo
#Elegimos borrar de prepo ceccor porque viendo como devuelve los resultados, da cosas que estan muy fuera de foco
methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 
            'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']
w, h = imgTemplate.shape[::-1]
for meth in methods:
    for img_rgb, img_gray in zip(imgsMessi, imgsMessiGray):
        # Hago una copia de la imagen porque ciclo a ciclo le dibujo rectángulos
        img_salida = img_rgb.copy()
        img_gray = gaussian_pyramid(img_gray, level)[selected]
        #Hacemos una imagen secundaria que sera reescalada pero que alli se le dibujaran los rectangulos
        other = gaussian_pyramid(img_salida, level)[selected]
        method = eval(meth)

        # Aplicamos la coincidencia de patrones
        #--------------------------------------
        res = cv.matchTemplate(img_gray,imgTemplate,method)

        # Encontramos los valores máximos y mínimos
        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)

        # Si el método es TM_SQDIFF o TM_SQDIFF_NORMED, tomamos el mínimo
        if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
            top_left = min_loc
        else:
            top_left = max_loc
        # Marcamos el lugar donde lo haya encontrado
        #----------------------------------------
        bottom_right = ((top_left[0] + w), (top_left[1] + h))
        cv.rectangle(other,top_left, bottom_right, 255, 2)
        ##Reescalamos:
        for i in range(level):
            other = cv.pyrUp(other)

        # Graficamos el procesamiento y la salida
        #----------------------------------------
        plt.figure()
        # Imagen original
        plt.subplot(122),plt.imshow(img_salida)
        plt.title('Original Image'), plt.xticks([]), plt.yticks([])
        # imagen recuadrada
        plt.subplot(121),plt.imshow(other)
        plt.title('Matching Result'), plt.xticks([]), plt.yticks([])



        plt.suptitle(meth)
        plt.show()

# Reconocimiento de caras

Debido a que los resultados anteriores no tuvieron mucho éxito se tomó un camino alternativo para poder encontrar la cara de Messi en las imágenes. Este camino consistió en encontrar las caras de las personas que aparecen en las imágenes y a estas compararlas con el patrón.

OpenCV nos ofrece clasificadores pre entrenados no solo de rostros de personas, sino de ojos, sonrisa, entre otros. Los clasificadores utilizados fueron los de Haar y se descargaron del siguiente link: https://github.com/opencv/opencv/tree/master/data/haarcascades

In [ ]:
#Volvemos a cargar todas las imagenes
pattern = None
patternGray = None
imgsMessi = []
imgsMessiGray = []
imgNames = glob('./Fotos/*') #Cargamos todas las imagenes que se encuentran en el folder 'Fotos'
for name in imgNames:
    img = cv.imread(name)
    imgGray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    if 'Template' in name:
        pattern = img
        patternGray = imgGray
    else:
        imgsMessi.append(img)
        imgsMessiGray.append(imgGray)

In [ ]:
def _getProportion(width, heigth, desiredWidth, desiredHeight):
    return width / desiredWidth, heigth / desiredHeight

def _getRes(img_gray, template):
    methods = ['cv.TM_CCOEFF']
    finalRes = None
    minValue = 0
    maxValue = 0
    for method in methods:
        method = eval(method)
        res = cv.matchTemplate(img_gray,template,method)
        actualMin, actualMax, _, _ = cv.minMaxLoc(res)
        if abs(maxValue - minValue) < abs(actualMax - actualMin):
            maxValue, minValue = actualMax, actualMin
            finalRes = res
    return res
        

def getMaxRes(faces, img, template, width = 100, heigth = 100):
    minValue = 0
    maxValue = 0
    face = None
    x, y = len(img[0]), len(img)
    for actualX,actualY, w, h in faces:
        imgToResize = img.copy()
        newSize = _getProportion(width, heigth, w, h)
        imgToResize = cv.resize(img, (int(x * newSize[0]), int(y * newSize[1])))
        actualRes = _getRes(imgToResize, template)
        actualMin, actualMax, _, _ = cv.minMaxLoc(actualRes)
        if abs(maxValue - minValue) < abs(actualMax - actualMin):
            maxValue, minValue = actualMax, actualMin
            face = (actualX,actualY,w,h)
    return face

In [ ]:
def removeUnnecessaryFaces(faces, grayImage):
    eyeCascade = cv.CascadeClassifier('haarcascade_eye.xml')
    okayFaces = []
    for face in faces:
        (x, y, w, h) = face
        roi_gray = grayImage[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eyeCascade.detectMultiScale(roi_gray, 1.1, 3)
        if len(eyes) >0:
            okayFaces.append(face)
    return okayFaces if okayFaces else faces

def faceRecognition(imgs, template):
    faceCascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
    _, _, templateWidth, templateHeigth = faceCascade.detectMultiScale(template)[0]
    for img in imgs:
        grayImage = img.copy()
        faces = faceCascade.detectMultiScale(grayImage, 1.1, 3)
        faces = removeUnnecessaryFaces(faces, grayImage)
        x,y,w,h = getMaxRes(faces, img, template, templateWidth, templateHeigth)
        cv.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        plotter(img, step = 100)

A continuación se describen que hacen las funciones de arriba:

+ **removeUnnecessaryFaces**: Al realizar el reconocimiento de caras de vez en cuando se obtienen resultados inesperados, por ej se pudo ver que el escudo en la camiseta era marcado como una cara, lo cual obviamente es incorrecto. Para solucionar esto se cargó un clasificador para los ojos y lo que se hizo fue preguntar si los ojos se encuentran dentro del cuadro que se dijo previamente que era una cara, en caso de ser así efectivamente se consideró que es una cara y en caso contrario se la descartó. Con esto se logró solucionar el problema de marcar partes de la imagen que no contenían una cara.

+ **faceRecognition**: Como su nombre lo indica esta función es la que se encarga de realizar el reconocimiento de caras. Esta función a su vez llama a otras funciones para comparar el patrón con las caras detectadas y marca con un rectángulo rojo la que más se asemeje a la del patrón.

+ **_getRes**: Con esta función lo que hacemos es comparar las caras detectadas con la del patrón. En caso de haber una alta coincidencia devolvemos en que lugar se encuentra la cara para que se marque el rectángulo rojo.

In [ ]:
faceRecognition(imgsMessi, pattern)

In [ ]:
def faceRecognition2(imgs):
    faceCascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
    eyeCascade = cv.CascadeClassifier('haarcascade_eye.xml')
    
    for img in imgs:
        grayImage = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
        faces = faceCascade.detectMultiScale(grayImage, 1.1, 3)
        for (x, y, w, h) in faces:
            roi_gray = grayImage[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
            eyes = eyeCascade.detectMultiScale(roi_gray, 1.1, 3)
            for (ex,ey,ew,eh) in eyes:
                cv.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
                cv.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
        plotter(img)

A continuación mostraremos las caras y ojos que el algoritmo marca al analizar cada una de las imágenes.

In [ ]:
faceRecognition2(imgsMessi)

## Conclusiones

Para realizar un análisis a distintas escalas es muy útil usar pirámides, pero no siempre se logra obtener los resultados que uno desea.

Los métodos que ya contienen algoritmos entrenados son muy eficientes tanto en tiempo de ejecución como también en detectar distintos elementos en una imagen (en nuestro caso caras y ojos). Sin embargo estos también a veces marcan resultados que son falsos positivos, pero realizando una buena combinación entre ellos se puede detectar caras correctamente.

Como se puede apreciar en las imágenes de arriba, casi en todas se logró detectar a Messi correctamente (6/7), la única que tuvo inconvenientes es la que él se encuentra en la playa dado que para ese caso los ojos no se detectaron y por lo tanto al no tenerlos su cara fue descartada por el algoritmo.